# Seasonal Climate Data Analysis

## M9 Challenge - finding a few key aspects of Oahu’s seasonal weather data

### Complete the following steps:

* Identify key statistical data in June across all of the stations and years using the describe() function.
* Identify key statistical data in December across all stations and years using the describe() function.
* Share your findings in the Jupyter Notebook with a few sentences describing the key differences in weather between June and December and 2-3 recommendations for further analysis.

### Key observations for Hawaii climate data in June and Dec 

#### Key diferences between the month of July and December (Challenge)

* The summary rain data shows descriptive stats for the June and Dec months. 
* In General for all stations the month of Dec has the highest recorded precipitation for all Stations
* The data for Honolulu shows up to 5 inches of rain in Dec as opposed to an avg of less than 1 inch in June.
* Station name MANOA LYON ARBO 785.2, HI US - has a very low change in average precipitaion betwen June and Dec (June=4.57 vs Dec=4.75 )

#### Further Analysis recommendations (Challenge)

* Get statistical summary for all months, quarters and seasons for each station
* Identify areas/stations with the most months having lowest rainfall averages
* Summarize areas where the weather would be more favoravle for the surf and icecream business based on low average rainfal, assuming that lower rainfal translates in to better business for the shops.

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect



In [2]:
# Create engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# reflect database into our classes
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [4]:
# explore
explore = inspect(engine)
explore.get_table_names()

['measurement', 'station']

In [5]:
# Get column names and types for measurement table
columns = explore.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [6]:
# Get column names and types for station table
columns = explore.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [7]:
# Save references to each table
Measurements = Base.classes.measurement
Station = Base.classes.station

In [8]:
# create a session link from Python to our database
session = Session(engine)

### Identify key statistical data in June across all of the stations and years using the describe() function.

In [9]:
# Query for June data, info and describe function
june_query = "SELECT id,station,date,prcp,tobs FROM measurement WHERE date like '%-06-%'"
june_df = pd.read_sql_query(june_query, engine)
#june_df.info()
#june_df.head(60)
june_df.describe()

,id,prcp,tobs
count,1700.000000,1574.000000,1700.000000
mean,9752.647647,0.136360,74.944118
std,5617.512522,0.335731,3.257417
min,134.000000,0.000000,64.000000
25%,5020.750000,0.000000,73.000000
50%,9673.500000,0.020000,75.000000
75%,14547.250000,0.120000,77.000000
max,19497.000000,4.430000,85.000000


### Identify key statistical data in December across all stations and years using the describe() function.

In [10]:
# Query for December data, info and describe
dec_query = "SELECT id,station,date,prcp,tobs FROM measurement WHERE date like '%-12-%'"
dec_df = pd.read_sql_query(dec_query, engine)
#dec_df.info()
#dec_df.head(60)
dec_df.describe()

,id,prcp,tobs
count,1517.000000,1405.000000,1517.000000
mean,9816.710613,0.216819,71.041529
std,5658.321092,0.541399,3.745920
min,306.000000,0.000000,56.000000
25%,4840.000000,0.000000,69.000000
50%,9876.000000,0.030000,71.000000
75%,14717.000000,0.150000,74.000000
max,19328.000000,6.420000,83.000000


#### Exported data to csv using following method:

``` shell (base) damianperreira-mbp:surfs_up damianperreira$ sqlite3
SQLite version 3.30.0 2019-10-04 15:03:17
Enter ".help" for usage hints.
Connected to a transient in-memory database.
Use ".open FILENAME" to reopen on a persistent database.
sqlite> .headers on
sqlite> .mode csv
sqlite> .output data.csv
sqlite> .open hawaii.sqlite
sqlite> select * from measurement;``` 

#### imported data to Postgresql tables with columns as text, then ran following sql view to get summary totals by station, for each month

``` mysql select s.name,s.elevation,(case when m.date like '%-06-%' then 'june' else 'dec' end) as groupings,
max(TO_NUMBER(m.prcp,'99G999D9S')) as max_prcp, --max prcp --
min(m.prcp) as min_prcp, --min prcp
avg(TO_NUMBER(m.prcp,'99G999D9S'))*10 as avg_prcp, --avg prcp
max(m.tobs) as max_tobs, --max obs
min(m.tobs) as min_tobs, --min obs
avg(TO_NUMBER(m.tobs,'99G999D9S'))*10 as avg_tobs --avg obs
from station s 
left join (select * from measurement where date like '%-06-%' or date like '%-12-%') m on (s.station = m.station ) 
group by 
s.name,s.elevation,(case when m.date like '%-06-%' then 'june' else 'dec' end)
order by 1 asc,3 desc;``` 

In [11]:
# exported the above query results to a csv file and creating a dataframe to view results
data = pd.read_csv("challenge_summary.csv") 
# Preview the jun and dec data for all stations
data

,station,elevation,month,max_prcp,min_prcp,avg_prcp,max_tobs,min_tobs,avg_tobs
0,"HONOLULU OBSERVATORY 702.2, HI US",0.9,june,0.5,0.00,0.069182,85,66,74.139394
1,"HONOLULU OBSERVATORY 702.2, HI US",0.9,dec,4.0,0.00,1.278146,81,57,69.684211
2,"KANEOHE 838.1, HI US",14.6,june,4.4,0.00,0.931624,83,68,74.050847
3,"KANEOHE 838.1, HI US",14.6,dec,4.9,0.00,1.777778,80,63,71.069444
4,"KUALOA RANCH HEADQUARTERS 886.9, HI US",7.0,june,3.6,0.00,0.880240,81,67,76.005376
5,"KUALOA RANCH HEADQUARTERS 886.9, HI US",7.0,dec,2.5,0.00,1.285714,79,62,73.224719
6,"MANOA LYON ARBO 785.2, HI US",152.4,june,2.8,0.00,4.579439,79,66,71.937220
7,"MANOA LYON ARBO 785.2, HI US",152.4,dec,6.4,0.00,4.751269,83,60,69.291262
8,"PEARL CITY, HI US",11.9,june,1.7,0.00,0.443038,82,64,76.655405
9,"PEARL CITY, HI US",11.9,dec,2.8,0.00,1.327273,82,64,71.834862
